В данной задаче требуется решить численно систему линейных алгебраических уравнений методом сопряженных градиентов. Текущий способ требует от исходной матрицы выполнения свойств симметричности и положительной определенности. Но даже если они не выполняются, то, согласно замечанию 1 конспекта, "систему можно привести к эквивалентному виду домножением обеих частей равенства на транспонированную матрицу ATAx=ATb". Для простоты будем генерировать сразу подходящие матрицы, хоть это и несколько сложнее.

In [1]:
import numpy as np
import pandas as pd
from math import sqrt

Зададим размер матриц.

In [2]:
N = 10

Напишем функцию решения СЛАУ методом сопряженных градиентов, используя условия малости невязки для остановки. А также близость двух приближений друг к другу (apx1, apx2).

In [3]:
def calc_b(A, p, g):
    ap = np.dot(A,p)
    return (np.dot(ap, g)) / np.dot(ap, p)
def calc_a(A, p, g):
    ap=np.dot(A, p)
    return -(np.dot(g, p) / np.dot(ap, p))
def calc_p(g, b, p):
    return -g + np.multiply(b, p)
def calc_g(A, B, x):
    return np.subtract(np.dot(A, x), b)
def solve(A, B, max_iterations=1000):
    n=len(A)
    x=B
    g=calc_g(A, B, x)
    p=g
    a=calc_a(A, p, g)
    it = 0
    while (np.linalg.norm(g) > 1e-10):
        xk = np.add(x, np.multiply(a, p))
        gk = calc_g(A, B, xk)
        b = calc_b(A, p, gk)
        pk = calc_p(gk, b, p)
        ak = calc_a(A, pk, gk)
        
        apx1 = np.linalg.norm(g) / np.linalg.norm(b)
        apx2 = np.linalg.norm(np.subtract(xk, x)) / np.linalg.norm(xk)
        if (np.allclose([apx1, apx2], [0, 0])):
            break
        
        x=xk
        g=gk
        p=pk
        a=ak
        it = it + 1
        if (it == max_iterations):
            print("Решение не было найдено за {0} шагов.".format(max_iterations))
            return x
    print("Решение найдено за {0} итераций.".format(it))    
    return x    

Напишем функцию сравнения библиотечного и авторского решения.

In [4]:
def compare_solutions(A, B, X):
    sol_b = np.dot(A, X)
    print("Решение является точным? {0}".format(np.allclose(sol_b, B)))

Загрузим из файлов матрицы, на которых мы будем проверять корректность реализации.

In [5]:
delimiter = ','
well = np.loadtxt('well.txt', delimiter=delimiter)
random = np.loadtxt('random.txt', delimiter=delimiter)
poor = np.loadtxt('poor.txt', delimiter=delimiter)
b = np.loadtxt('right.txt', delimiter=delimiter)

## Решение хорошо обусловленной матрицы.

In [6]:
well_x = solve(well, b)
compare_solutions(well, b, well_x)

Решение найдено за 9 итераций.
Решение является точным? True


## Решение рандомной матрицы.

In [7]:
random_x = solve(random, b)
compare_solutions(random, b, random_x)

Решение найдено за 11 итераций.
Решение является точным? True


_Вывод: Зачастую для нее требуется больше чем n шагов из-за накопленной машинной погрешности._

## Решение плохо обусловленной матрицы.

In [8]:
poor_x = solve(poor, b)
compare_solutions(poor, b, poor_x)

Решение не было найдено за 1000 шагов.
Решение является точным? False


_Вывод: К сожалению, для данной матрицы решение не удается найти точное решение за обозримое количество времени в силу связи погрешности и обусловленности._

Теперь сохраним наши результаты в таблицу.

In [9]:
gradient_df = pd.DataFrame({'well': well_x, 'random': random_x, 'poor': poor_x})

In [10]:
gradient_df.to_csv('gradient.csv', index=None)